In [10]:
from utils.custom_dataset import YOLO6ChannelDataset

train_dataset = YOLO6ChannelDataset(
    left_img_dir="D:/intezet/Bogi/Yolo/data/images/train/left",
    right_img_dir="D:/intezet/Bogi/Yolo/data/images/train/right",
    label_dir="D:/intezet/Bogi/Yolo/data/labels/train"
)

val_dataset = YOLO6ChannelDataset(
    left_img_dir="D:/intezet/Bogi/Yolo/data/images/val/left",
    right_img_dir="D:/intezet/Bogi/Yolo/data/images/val/right",
    label_dir="D:/intezet/Bogi/Yolo/data/labels/val"
)

In [ ]:
import torch
from torch.utils.data import DataLoader
from ultralytics.nn.tasks import DetectionModel
from types import SimpleNamespace
from tqdm import tqdm
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 50
batch_size = 8
save_path = "yolov8_custom6.pt"


# --- Collate Function ---
def collate_fn(batch):
    imgs, targets = zip(*batch)
    imgs = torch.stack(imgs, dim=0)  # shape: (batch, 6, H, W)
    # targets is a tuple of tensors [num_boxes_i, 5], keep as list for model.train()
    targets = list(targets)
    return imgs, targets



train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          collate_fn=collate_fn, num_workers=4)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_fn, num_workers=4)


# --- Model ---
model = DetectionModel(cfg='ultralytics/cfg/models/v8/yolov8n.yaml', ch=6, nc=6).to(device)
model.args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5, reg_max=15)
model.criterion = model.init_criterion()
model.to(device)
# print(f"Expected output channels: {(model.args.reg_max + 1) * 4 + model.nc}")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# --- Training Loop ---
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    
    for imgs, targets in pbar:
        # print(f"imgs.shape: {imgs.shape}")
        imgs = imgs.to(device)
        targets = targets.to(device)

        if targets.shape[0] == 0:
            continue

        optimizer.zero_grad()
        preds = model(imgs)

        batch = {
            "img": imgs,
            "batch_idx": targets[:, 0],
            "cls": targets[:, 1],
            "bboxes": targets[:, 2:]
        }
        
        loss, loss_items = model.loss(batch, preds)  # loss is scalar, loss_items is [box, cls, dfl]
        loss = loss.sum()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"📘 Epoch {epoch+1} - Train Loss: {avg_loss:.4f}")

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs = imgs.to(device)
            targets = targets.to(device)

            if targets.shape[0] == 0:
                continue

            preds = model(imgs)
            batch = {
                "img": imgs,
                "batch_idx": targets[:, 0],
                "cls": targets[:, 1],
                "bboxes": targets[:, 2:]
            }
            
            loss_out, loss_out_items = model.loss(batch, preds)  # loss is scalar, loss_items is [box, cls, dfl]
            loss_out = loss_out.sum()
            val_loss += loss_out.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"🧪 Epoch {epoch+1} - Val Loss: {avg_val_loss:.4f}")

    # --- Save ---
    if (epoch + 1) % 5 == 0 or (epoch + 1) == epochs:
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved to: {save_path}")




New https://pypi.org/project/ultralytics/8.3.160 available  Update with 'pip install -U ultralytics'


SyntaxError: '[31m[1mdataloader[0m' is not a valid YOLO argument. 
'[31m[1mval_dataloader[0m' is not a valid YOLO argument. 

    Arguments received: ['yolo', '--f=c:\\Users\\etiko\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3828b9ccdfe2b03f2c85fd307ab23270c974989ba.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ['segment', 'classify', 'detect', 'pose', 'obb']
                MODE (required) is one of ['val', 'train', 'predict', 'export', 'track', 'benchmark']
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at ima

In [ ]:
import torch
from ultralytics.nn.tasks import DetectionModel
from types import SimpleNamespace

# Configuration
weights_path = "yolov8_custom6.pt"  # Path to your trained model weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your 6-channel YOLOv8 model with 6 classes
model = DetectionModel(cfg='ultralytics/cfg/models/v8/yolov8.yaml', ch=6, nc=6)
model.args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5, reg_max=15)
model.criterion = model.init_criterion()
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()
model.to(device)

In [13]:
from torchvision import transforms as T
from PIL import Image

transform = T.Compose([
    T.Resize((640, 640)),  # Adjust if your model uses a different size
    T.ToTensor()
])

def load_6ch_image(path_left, path_right):
    img_right = transform(Image.open(path_right).convert("RGB"))  # (3, H, W)
    img_left = transform(Image.open(path_left).convert("RGB"))  # (3, H, W)
    return torch.cat([img_right, img_left], dim=0)  # (6, H, W)

In [14]:
import torch
import cv2
from ultralytics.nn.tasks import DetectionModel
from ultralytics.utils.ops import non_max_suppression

# Prediction function
def predict_and_draw(img_path_right, img_path_left, output_path, conf_thresh=0.1, iou_thresh=0.65):
    # Prepare 6-channel image
    img_6ch = load_6ch_image(img_path_right, img_path_left)  # Tensor (6, H, W)
    img_6ch = img_6ch.unsqueeze(0).to(device)  # (1, 6, H, W)

    with torch.no_grad():
        raw_preds = model(img_6ch)[0]  # Get raw output
        
        # print("Raw preds shape:", raw_preds.shape)  # Expect: [1, N, 6+num_classes]
        # print("Raw preds min/max:", raw_preds.min().item(), raw_preds.max().item())
        raw_preds[..., 4:] = raw_preds[..., 4:].sigmoid()
        
        # print("Raw preds shape afer sigomid:", raw_preds.shape)  # Expect: [1, N, 6+num_classes]
        # print("Raw preds min/max after sigmoid:", raw_preds.min().item(), raw_preds.max().item())
        
        preds = non_max_suppression(raw_preds, conf_thres=conf_thresh, iou_thres=iou_thresh, nc=6)[0]
        
        # print("Filtered boxes:", preds.shape)
        # print(preds)


    original = cv2.imread(img_path_right)
    h, w = original.shape[:2]  # Get image dimensions

    if preds is not None and len(preds):
        for *xyxy, conf, cls in preds:
            # Unpack and scale to original image size
            x1, y1, x2, y2 = xyxy
            x1 = int(x1 * w)
            y1 = int(y1 * h)
            x2 = int(x2 * w)
            y2 = int(y2 * h)

            class_names = ['EUK', 'FC', 'FE ', 'class3', 'class4', 'class5']
            label = f"{class_names[int(cls.item())]}: {conf:.2f}"

            cv2.rectangle(original, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(original, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 255, 0), 1, cv2.LINE_AA)


    cv2.imwrite(output_path, original)

In [15]:
input_folder = "Yolo/data/test_for_prediction"
output_folder = "Yolo/runs/predict_custom_6ch_v8"
os.makedirs(output_folder, exist_ok=True)

file_names = [f for f in os.listdir(input_folder + "/right") if ".png" in f]
i = 1


for file_name in file_names:
    
    right_path = os.path.join(input_folder + "/right", file_name)
    left_path = os.path.join(input_folder + "/left", file_name)
    output_path = os.path.join(output_folder, file_name.replace(".png", "_pred.png"))

    predict_and_draw(right_path, left_path, output_path)
    print(f"Saved: {output_path} {i}/{len(file_names)}")
    i += 1

IndexError: The shape of the mask [80, 80] at index 0 does not match the shape of the indexed tensor [70, 80, 80] at index 0

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n_6ch.yaml')

model.train(data='dataset.yaml', epochs=50, batch=8, imgsz=640)